In [ ]:
from __future__ import print_function, division
import numpy as np
import tensorflow as tf

% matplotlib inline
import matplotlib.pyplot as plt

In [2]:
tf.__version__

In [3]:
num_epochs = 5
total_series_length = 50000
num_steps = 15
state_size = 4
num_classes = 2
batch_size = 5
num_batches = total_series_length//batch_size//num_steps

num_layers = 3

echo_step = 3

In [4]:
def generateData():
    x = np.array(np.random.choice(2, total_series_length, p=[0.5, 0.5]))
    y = np.roll(x, echo_step)
    y[0:echo_step] = 0

    x = x.reshape((batch_size, -1))  # The first index changing slowest, subseries as rows
    y = y.reshape((batch_size, -1))

    return (x, y)

In [5]:
batchX_placeholder = tf.placeholder(tf.float32, [batch_size, num_steps])
batchY_placeholder = tf.placeholder(tf.int32, [batch_size, num_steps])

# init_state = tf.placeholder(tf.float32, [batch_size, state_size])

# < 4
init_state = tf.placeholder(tf.float32, [num_layers, 2, batch_size, state_size])
state_per_layer_list = tf.unstack(init_state, axis=0)
rnn_tuple_state = tuple(
    [tf.nn.rnn_cell.LSTMStateTuple(state_per_layer_list[idx][0], state_per_layer_list[idx][1])
     for idx in range(num_layers)]
)
# > 4

# < 3
# cell_state = tf.placeholder(tf.float32, [batch_size, state_size])
# hidden_state = tf.placeholder(tf.float32, [batch_size, state_size])
# init_state = tf.nn.rnn_cell.LSTMStateTuple(cell_state, hidden_state)
# > 3

In [6]:
W = tf.Variable(np.random.rand(state_size+1, state_size), dtype=tf.float32)
b = tf.Variable(np.zeros((1,state_size)), dtype=tf.float32)

W2 = tf.Variable(np.random.rand(state_size, num_classes),dtype=tf.float32)
b2 = tf.Variable(np.zeros((1,num_classes)), dtype=tf.float32)

In [ ]:
# Unpack columns
inputs_series = tf.unstack(batchX_placeholder, axis=1)
labels_series = tf.unstack(batchY_placeholder, axis=1)

In [ ]:
# Forward pass
current_state = init_state
states_series = []
for current_input in inputs_series:
    current_input = tf.reshape(current_input, [batch_size, 1])
    input_and_state_concatenated = tf.concat([current_input, current_state], axis=1)  # Increasing number of columns

    next_state = tf.tanh(tf.matmul(input_and_state_concatenated, W) + b)  # Broadcasted addition
    states_series.append(next_state)
    current_state = next_state

In [7]:
# Unpack columns
inputs_series = tf.split(batchX_placeholder, num_steps, axis=1)
labels_series = tf.unstack(batchY_placeholder, axis=1)

# Forward passes

# < 2
# cell = tf.contrib.rnn.BasicRNNCell(state_size)
# states_series, current_state = tf.contrib.rnn.static_rnn(cell, inputs_series, init_state)
# > 2

# < 3
# cell = tf.contrib.rnn.BasicLSTMCell(state_size, state_is_tuple=True)
# states_series, current_state = tf.contrib.rnn.static_rnn(cell, inputs_series, init_state)
# > 3

# < 4
# cell = tf.contrib.rnn.BasicLSTMCell(state_size, state_is_tuple=True)
# cell = tf.contrib.rnn.MultiRNNCell([cell] * num_layers, state_is_tuple=True)
# states_series, current_state = tf.contrib.rnn.static_rnn(cell, inputs_series, initial_state=rnn_tuple_state)
# > 4

# < 5
cell = tf.contrib.rnn.BasicLSTMCell(state_size, state_is_tuple=True)
cell = tf.contrib.rnn.MultiRNNCell([cell] * num_layers, state_is_tuple=True)
states_series, current_state = tf.nn.dynamic_rnn(cell, tf.expand_dims(batchX_placeholder, -1), initial_state=rnn_tuple_state)
states_series = tf.reshape(states_series, [-1, state_size])
# > 5

ValueError: Dimensions must be equal, but are 8 and 5 for 'rnn/while/rnn/multi_rnn_cell/cell_0/basic_lstm_cell/MatMul_1' (op: 'MatMul') with input shapes: [5,8], [5,16].

In [ ]:
# < 1-4
# logits_series = [tf.matmul(state, W2) + b2 for state in states_series] #Broadcasted addition
# predictions_series = [tf.nn.softmax(logits) for logits in logits_series]

# losses = [tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=labels) for logits, labels in zip(logits_series,labels_series)]
# > 1-4

# < 5
logits = tf.matmul(states_series, W2) + b2 #Broadcasted addition
labels = tf.reshape(batchY_placeholder, [-1])

logits_series = tf.unpack(tf.reshape(logits, [batch_size, truncated_backprop_length, num_classes]), axis=1)
predictions_series = [tf.nn.softmax(logit) for logit in logits_list]

losses = tf.nn.sparse_softmax_cross_entropy_with_logits(logits, labels)
# > 5

total_loss = tf.reduce_mean(losses)

train_step = tf.train.AdagradOptimizer(0.3).minimize(total_loss)

In [ ]:
def plot(loss_list, predictions_series, batchX, batchY):
    plt.subplot(2, 3, 1)
    plt.cla()
    plt.plot(loss_list)

    for batch_series_idx in range(5):
        one_hot_output_series = np.array(predictions_series)[:, batch_series_idx, :]
        single_output_series = np.array([(1 if out[0] < 0.5 else 0) for out in one_hot_output_series])

        plt.subplot(2, 3, batch_series_idx + 2)
        plt.cla()
        plt.axis([0, num_steps, 0, 2])
        left_offset = range(num_steps)
        plt.bar(left_offset, batchX[batch_series_idx, :], width=1, color="blue")
        plt.bar(left_offset, batchY[batch_series_idx, :] * 0.5, width=1, color="red")
        plt.bar(left_offset, single_output_series * 0.3, width=1, color="green")

    plt.draw()
    plt.pause(0.0001)

In [ ]:
with tf.Session() as sess:
    sess.run(tf.initialize_all_variables())
    plt.ion()
    plt.figure()
    plt.show()
    loss_list = []

    for epoch_idx in range(num_epochs):
        x,y = generateData()
        # _current_state = np.zeros((batch_size, state_size))
        
        # < 3
        # _current_cell_state = np.zeros((batch_size, state_size))
        # _current_hidden_state = np.zeros((batch_size, state_size))
        # > 3
        
        # < 4
        _current_state = np.zeros((num_layers, 2, batch_size, state_size))
        # < 4

        print("New data, epoch", epoch_idx)

        for batch_idx in range(num_batches):
            start_idx = batch_idx * num_steps
            end_idx = start_idx + num_steps

            batchX = x[:,start_idx:end_idx]
            batchY = y[:,start_idx:end_idx]

            # < 1,2,4
            _total_loss, _train_step, _current_state, _predictions_series = sess.run(
                [total_loss, train_step, current_state, predictions_series],
                feed_dict={
                    batchX_placeholder:batchX,
                    batchY_placeholder:batchY,
                    init_state:_current_state
                })
            # > 1,2,4
            
            # < 3
            # _total_loss, _train_step, _current_state, _predictions_series = sess.run(
            #     [total_loss, train_step, current_state, predictions_series],
            #     feed_dict={
            #         batchX_placeholder: batchX,
            #         batchY_placeholder: batchY,
            #         cell_state: _current_cell_state,
            #         hidden_state: _current_hidden_state
            #    })
            # 
            # _current_cell_state, _current_hidden_state = _current_state
            # > 3
            
            loss_list.append(_total_loss)

            if batch_idx%100 == 0:
                print("Step",batch_idx, "Loss", _total_loss)
                plot(loss_list, _predictions_series, batchX, batchY)

plt.ioff()
plt.show()